# Lab 12: Customer Management APIs

## Learning Objectives
- Implement complete CRUD operations for customers
- Create customer search endpoints with filtering
- Implement pagination for large datasets
- Build customer listing APIs with role-based access

**Duration:** 45 minutes  
**Prerequisites:** Completion of Authentication and Security

## Step 1: Customer CRUD Operations

Implement create, read, update, and delete operations for customer management.

In [ ]:
# Cell 7: Customer management API endpoints
@app.post("/customers", response_model=CustomerResponse, tags=["Customer Management"])
async def create_customer(
    customer_data: CustomerCreate,
    current_user: Dict[str, Any] = Depends(require_role(UserRole.AGENT))
):
    """Create new customer"""
    
    # Check if customer with email already exists
    check_query = """
    MATCH (c:Customer {email: $email})
    RETURN c.customer_id as customer_id
    """
    
    existing = connection_manager.execute_query(check_query, {"email": customer_data.email})
    if existing:
        raise HTTPException(
            status_code=status.HTTP_409_CONFLICT,
            detail="Customer with this email already exists"
        )
    
    # Create new customer
    customer_id = f"CUST_{secrets.token_hex(6).upper()}"
    
    create_query = """
    CREATE (c:Customer {
        customer_id: $customer_id,
        first_name: $first_name,
        last_name: $last_name,
        email: $email,
        phone: $phone,
        date_of_birth: date($date_of_birth),
        address: $address,
        city: $city,
        state: $state,
        zip_code: $zip_code,
        customer_since: datetime(),
        created_by: $created_by,
        risk_score: 0.5
    })
    RETURN c
    """
    
    customer_dict = customer_data.dict()
    customer_dict.update({
        "customer_id": customer_id,
        "created_by": current_user["sub"],
        "date_of_birth": customer_data.date_of_birth.isoformat()
    })
    
    result = connection_manager.execute_write_query(create_query, customer_dict)
    
    if not result:
        raise HTTPException(
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail="Failed to create customer"
        )
    
    # Return created customer with additional stats
    return_query = """
    MATCH (c:Customer {customer_id: $customer_id})
    OPTIONAL MATCH (c)-[:HAS_POLICY]->(p:Policy)
    RETURN c.customer_id as customer_id,
           c.first_name as first_name,
           c.last_name as last_name, 
           c.email as email,
           c.phone as phone,
           c.date_of_birth as date_of_birth,
           c.address as address,
           c.city as city,
           c.state as state,
           c.zip_code as zip_code,
           c.customer_since as customer_since,
           c.risk_score as risk_score,
           count(p) as total_policies,
           coalesce(sum(p.premium_amount), 0.0) as total_premium
    """
    
    result = connection_manager.execute_query(return_query, {"customer_id": customer_id})
    return CustomerResponse(**result[0])

@app.get("/customers/{customer_id}", response_model=CustomerResponse, tags=["Customer Management"])
async def get_customer(
    customer_id: str,
    current_user: Dict[str, Any] = Depends(get_current_user)
):
    """Get customer by ID"""
    
    query = """
    MATCH (c:Customer {customer_id: $customer_id})
    OPTIONAL MATCH (c)-[:HAS_POLICY]->(p:Policy {status: 'Active'})
    RETURN c.customer_id as customer_id,
           c.first_name as first_name,
           c.last_name as last_name,
           c.email as email,
           c.phone as phone,
           c.date_of_birth as date_of_birth,
           c.address as address,
           c.city as city,
           c.state as state,
           c.zip_code as zip_code,
           c.customer_since as customer_since,
           c.risk_score as risk_score,
           count(p) as total_policies,
           coalesce(sum(p.premium_amount), 0.0) as total_premium
    """
    
    result = connection_manager.execute_query(query, {"customer_id": customer_id})
    
    if not result:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Customer not found"
        )
    
    return CustomerResponse(**result[0])

print("✓ Customer CRUD operations configured")

## Step 2: Customer Listing with Pagination

Implement customer listing with search, filtering, and pagination support.

In [ ]:
@app.get("/customers", response_model=PaginatedResponse, tags=["Customer Management"])
async def list_customers(
    page: int = 1,
    per_page: int = 10,
    search: Optional[str] = None,
    state: Optional[str] = None,
    current_user: Dict[str, Any] = Depends(require_role(UserRole.AGENT))
):
    """List customers with pagination and filtering"""
    
    # Build WHERE clause for filtering
    where_conditions = []
    params = {}
    
    if search:
        where_conditions.append(
            "(toLower(c.first_name) CONTAINS toLower($search) OR "
            "toLower(c.last_name) CONTAINS toLower($search) OR "
            "toLower(c.email) CONTAINS toLower($search))"
        )
        params["search"] = search
    
    if state:
        where_conditions.append("c.state = $state")
        params["state"] = state
    
    where_clause = "WHERE " + " AND ".join(where_conditions) if where_conditions else ""
    
    # Count total customers
    count_query = f"""
    MATCH (c:Customer)
    {where_clause}
    RETURN count(c) as total
    """
    
    total_result = connection_manager.execute_query(count_query, params)
    total = total_result[0]["total"]
    
    # Get paginated customers
    skip = (page - 1) * per_page
    params.update({"skip": skip, "limit": per_page})
    
    list_query = f"""
    MATCH (c:Customer)
    {where_clause}
    OPTIONAL MATCH (c)-[:HAS_POLICY]->(p:Policy {{'status': 'Active'}})
    RETURN c.customer_id as customer_id,
           c.first_name as first_name,
           c.last_name as last_name,
           c.email as email,
           c.phone as phone,
           c.date_of_birth as date_of_birth,
           c.address as address,
           c.city as city,
           c.state as state,
           c.zip_code as zip_code,
           c.customer_since as customer_since,
           c.risk_score as risk_score,
           count(p) as total_policies,
           coalesce(sum(p.premium_amount), 0.0) as total_premium
    ORDER BY c.customer_since DESC
    SKIP $skip LIMIT $limit
    """
    
    result = connection_manager.execute_query(list_query, params)
    customers = [CustomerResponse(**record) for record in result]
    
    return PaginatedResponse(
        items=customers,
        total=total,
        page=page,
        per_page=per_page,
        pages=(total + per_page - 1) // per_page
    )

print("✓ Customer listing with pagination configured")

## Step 3: Customer Update Operations

Implement customer update functionality with validation and audit tracking.

In [ ]:
@app.put("/customers/{customer_id}", response_model=CustomerResponse, tags=["Customer Management"])
async def update_customer(
    customer_id: str,
    update_data: CustomerUpdate,
    current_user: Dict[str, Any] = Depends(require_role(UserRole.AGENT))
):
    """Update customer information"""
    
    # Check if customer exists
    check_query = """
    MATCH (c:Customer {customer_id: $customer_id})
    RETURN c.customer_id as customer_id
    """
    
    existing = connection_manager.execute_query(check_query, {"customer_id": customer_id})
    if not existing:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Customer not found"
        )
    
    # Build SET clause for updates
    update_fields = {k: v for k, v in update_data.dict().items() if v is not None}
    
    if not update_fields:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="No valid fields to update"
        )
    
    # Create SET clause
    set_clauses = [f"c.{field} = ${field}" for field in update_fields.keys()]
    set_clause = "SET " + ", ".join(set_clauses)
    set_clause += ", c.last_updated = datetime(), c.updated_by = $updated_by"
    
    update_fields.update({
        "customer_id": customer_id,
        "updated_by": current_user["sub"]
    })
    
    update_query = f"""
    MATCH (c:Customer {{customer_id: $customer_id}})
    {set_clause}
    RETURN c
    """
    
    connection_manager.execute_write_query(update_query, update_fields)
    
    # Return updated customer
    return await get_customer(customer_id, current_user)

print("✓ Customer management API endpoints configured")
print("✓ CRUD operations with validation and pagination ready")